In [51]:
import requests
import mysql.connector
from datetime import datetime
from itertools import compress
import numpy as np
import urllib.parse


def fetch_query_results(cur, numrecords=10):
    result = []
    if type(numrecords) == int:
        i = 0
        for row in cur:    
            if i < numrecords:
                result.append(row)
                i += 1
    elif numrecords == 'all':
        for row in cur:    
            result.append(row)
    return result



def retrieve_journals():    
    cnx = mysql.connector.connect(user='root', password='CarM5500',
                              host='localhost',
                              database='OA')
    curA = cnx.cursor()
    retrieve_journals_query = '''select name from doaj_journals order by doaj_seal desc, name asc;'''
    curA.execute(retrieve_journals_query)
    return fetch_query_results(curA, 'all')



def retrieve_article_bib_details(bibjson):
    try:
        pages = bibjson['start_page'] + '-' + bibjson['end_page']
    except:
        pages = ''
    try:
        title = str(bibjson['title'])
    except:
        title = ''
    try:
        year = bibjson['year']
    except:
        year = ''
    try:
        journal_title = bibjson['journal']['title']
    except:
        journal_title = ''
    try:
        journal_number = bibjson['journal']['number']
    except:
        journal_number = ''
    try:
        journal_volume = bibjson['journal']['volume']
    except:
        journal_volume = ''
    try:
        journal_publisher = bibjson['journal']['publisher']
    except:
        journal_publisher = ''
    try:
        link = bibjson['link'][0]['url']
    except:
        link = ''
    try:
        link_type = bibjson['link'][0]['type']
    except:
        link_type = ''
        
    authors = []
    try:
        for author in bibjson['author']:
            authors.append( author['name'] )
    except:
        pass
        
    subjects = []
    try:
        for subject in bibjson['subject']:
            subjects.append( subject['term'] )
    except:
        pass    

    return  {'title':title, 'pages':pages, 'year':year, 
             'journal_title':journal_title, 'journal_publisher': journal_publisher,
             'authors':str(authors), 'subjects':str(subjects)}



def retrieve_article_details(article):
    doaj_created_date = datetime.strptime( article['created_date'], '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')
    doaj_last_updated = datetime.strptime( article['last_updated'], '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')
    article_details = {'id':article['id'], 
                       'doaj_created_date':doaj_created_date,
                      'doaj_last_updated':doaj_last_updated}
    return { **article_details, **retrieve_article_bib_details(article['bibjson']) }


#########
def get_page_articles(page):
    try:
        print( 'Retrieving articles from page %s' % page['page'] )
        page_articles = []
        for article in page['results']:
            page_articles.append( retrieve_article_details( article ) )
        return page_articles
    except:
        pass

    
def get_data(journal_url):
    page = requests.get(journal_url).json()
    articles = get_page_articles(page)
    more_pages = True
    while more_pages:
        try:
            page = requests.get( page['next'] ).json()
            articles.extend( get_page_articles(page) )
        except:
            more_pages = False
    return articles



def write_articles_to_doaj_articles_table(articles):
    
    cnx = mysql.connector.connect(user='root', password='CarM5500',
                              host='localhost', charset = 'utf8mb4', collation='utf8mb4_unicode_ci',
                              database='OA')
    for article in articles:
        column_names        = article.keys()
        column_non_missing  = [] 
        for value in article.values():
            column_non_missing.append( value != '')

        non_missing_columns = list(compress(column_names, column_non_missing))
        non_missing_data    = list(compress(article.values(), column_non_missing))

        # concatenate into single string all the column names
        col_name_tuple = "("
        for column in non_missing_columns:
            col_name_tuple += column + ', '
        col_name_tuple = col_name_tuple[:-2] + ')'

        query = '''
        insert into doaj_articles %s
        values %s
        ''' % ( col_name_tuple, str(tuple(non_missing_data)) )

        cnx.cmd_query(query)
        cnx.commit()
    cnx.close()
    
    
    
def create_doaj_articles_table():
    query = '''
    create table doaj_articles
    (
    %s varchar(300) CHARACTER SET utf8,

    %s DATETIME,
    %s DATETIME,

    %s text CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,

    %s varchar(300) CHARACTER SET utf8,
    %s varchar(300) CHARACTER SET utf8,
    %s varchar(300) CHARACTER SET utf8,
    %s varchar(300) CHARACTER SET utf8,

    %s text CHARACTER SET utf8,
    %s text CHARACTER SET utf8
    );
    ''' % tuple(['id', 'doaj_created_date', 'doaj_last_updated', 
                'title', 'pages', 'year', 'journal_title', 'journal_publisher', 
                'authors', 'subjects'])

    cnx = mysql.connector.connect(user='root', password='CarM5500',
                              host='localhost', charset = 'utf8mb4', collation='utf8mb4_unicode_ci',
                              database='OA')
    cnx.cmd_query('drop table if exists doaj_articles;')
    cnx.cmd_query(query)
    cnx.commit();
    cnx.close();



# create_doaj_articles_table()
journals = retrieve_journals()

for journal in journals[3438]:
    print('Working on %s' % journal)
    journal_url = 'https://doaj.org/api/v1/search/articles/journal:"%s"' % urllib.parse.quote_plus(journal[0])
    articles = get_data( journal_url )
    write_articles_to_doaj_articles_table(articles)




Working on Educate~
Retrieving articles from page 1
Retrieving articles from page 2
Retrieving articles from page 3
Retrieving articles from page 4
Retrieving articles from page 5
Retrieving articles from page 6
Retrieving articles from page 7
Retrieving articles from page 8
Retrieving articles from page 9
Retrieving articles from page 10
Retrieving articles from page 11
Retrieving articles from page 12
Retrieving articles from page 13
Retrieving articles from page 14
Retrieving articles from page 15
Retrieving articles from page 16
Retrieving articles from page 17
Retrieving articles from page 18
Retrieving articles from page 19
Retrieving articles from page 20
Retrieving articles from page 21
Retrieving articles from page 22
Retrieving articles from page 23
Retrieving articles from page 24
Retrieving articles from page 25
Retrieving articles from page 26
Retrieving articles from page 27
Retrieving articles from page 28
Retrieving articles from page 29
Retrieving articles from page 30

Retrieving articles from page 245
Retrieving articles from page 246
Retrieving articles from page 247
Retrieving articles from page 248
Retrieving articles from page 249
Retrieving articles from page 250
Retrieving articles from page 251
Retrieving articles from page 252
Retrieving articles from page 253
Retrieving articles from page 254
Retrieving articles from page 255
Retrieving articles from page 256
Retrieving articles from page 257
Retrieving articles from page 258
Retrieving articles from page 259
Retrieving articles from page 260
Retrieving articles from page 261
Retrieving articles from page 262
Retrieving articles from page 263
Retrieving articles from page 264
Retrieving articles from page 265
Retrieving articles from page 266
Retrieving articles from page 267
Retrieving articles from page 268
Retrieving articles from page 269
Retrieving articles from page 270
Retrieving articles from page 271
Retrieving articles from page 272
Retrieving articles from page 273
Retrieving art

Retrieving articles from page 486
Retrieving articles from page 487
Retrieving articles from page 488
Retrieving articles from page 489
Retrieving articles from page 490
Retrieving articles from page 491
Retrieving articles from page 492
Retrieving articles from page 493
Retrieving articles from page 494
Retrieving articles from page 495
Retrieving articles from page 496
Retrieving articles from page 497
Retrieving articles from page 498
Retrieving articles from page 499
Retrieving articles from page 500
Retrieving articles from page 501
Retrieving articles from page 502
Retrieving articles from page 503
Retrieving articles from page 504
Retrieving articles from page 505
Retrieving articles from page 506
Retrieving articles from page 507
Retrieving articles from page 508
Retrieving articles from page 509
Retrieving articles from page 510
Retrieving articles from page 511
Retrieving articles from page 512
Retrieving articles from page 513
Retrieving articles from page 514
Retrieving art

Retrieving articles from page 727
Retrieving articles from page 728
Retrieving articles from page 729
Retrieving articles from page 730
Retrieving articles from page 731
Retrieving articles from page 732
Retrieving articles from page 733
Retrieving articles from page 734
Retrieving articles from page 735
Retrieving articles from page 736
Retrieving articles from page 737
Retrieving articles from page 738
Retrieving articles from page 739
Retrieving articles from page 740
Retrieving articles from page 741
Retrieving articles from page 742
Retrieving articles from page 743
Retrieving articles from page 744
Retrieving articles from page 745
Retrieving articles from page 746
Retrieving articles from page 747
Retrieving articles from page 748
Retrieving articles from page 749
Retrieving articles from page 750
Retrieving articles from page 751
Retrieving articles from page 752
Retrieving articles from page 753
Retrieving articles from page 754
Retrieving articles from page 755
Retrieving art

Retrieving articles from page 968
Retrieving articles from page 969
Retrieving articles from page 970
Retrieving articles from page 971
Retrieving articles from page 972
Retrieving articles from page 973
Retrieving articles from page 974
Retrieving articles from page 975
Retrieving articles from page 976
Retrieving articles from page 977
Retrieving articles from page 978
Retrieving articles from page 979
Retrieving articles from page 980
Retrieving articles from page 981
Retrieving articles from page 982
Retrieving articles from page 983
Retrieving articles from page 984
Retrieving articles from page 985
Retrieving articles from page 986
Retrieving articles from page 987
Retrieving articles from page 988
Retrieving articles from page 989
Retrieving articles from page 990
Retrieving articles from page 991
Retrieving articles from page 992
Retrieving articles from page 993
Retrieving articles from page 994
Retrieving articles from page 995
Retrieving articles from page 996
Retrieving art

...

Retrieving articles from page 1908
Retrieving articles from page 1909
Retrieving articles from page 1910
Retrieving articles from page 1911
Retrieving articles from page 1912
Retrieving articles from page 1913
Retrieving articles from page 1914
Retrieving articles from page 1915
Retrieving articles from page 1916
Retrieving articles from page 1917
Retrieving articles from page 1918
Retrieving articles from page 1919
Retrieving articles from page 1920
Retrieving articles from page 1921
Retrieving articles from page 1922
Retrieving articles from page 1923
Retrieving articles from page 1924
Retrieving articles from page 1925
Retrieving articles from page 1926
Retrieving articles from page 1927
Retrieving articles from page 1928
Retrieving articles from page 1929
Retrieving articles from page 1930
Retrieving articles from page 1931
Retrieving articles from page 1932
Retrieving articles from page 1933
Retrieving articles from page 1934
Retrieving articles from page 1935
Retrieving articles 

Retrieving articles from page 2143
Retrieving articles from page 2144
Retrieving articles from page 2145
Retrieving articles from page 2146
Retrieving articles from page 2147
Retrieving articles from page 2148
Retrieving articles from page 2149
Retrieving articles from page 2150
Retrieving articles from page 2151
Retrieving articles from page 2152
Retrieving articles from page 2153
Retrieving articles from page 2154
Retrieving articles from page 2155
Retrieving articles from page 2156
Retrieving articles from page 2157
Retrieving articles from page 2158
Retrieving articles from page 2159
Retrieving articles from page 2160
Retrieving articles from page 2161
Retrieving articles from page 2162
Retrieving articles from page 2163
Retrieving articles from page 2164
Retrieving articles from page 2165
Retrieving articles from page 2166
Retrieving articles from page 2167
Retrieving articles from page 2168
Retrieving articles from page 2169
Retrieving articles from page 2170
Retrieving articles 

Retrieving articles from page 2378
Retrieving articles from page 2379
Retrieving articles from page 2380
Retrieving articles from page 2381
Retrieving articles from page 2382
Retrieving articles from page 2383
Retrieving articles from page 2384
Retrieving articles from page 2385
Retrieving articles from page 2386
Retrieving articles from page 2387
Retrieving articles from page 2388
Retrieving articles from page 2389
Retrieving articles from page 2390
Retrieving articles from page 2391
Retrieving articles from page 2392
Retrieving articles from page 2393
Retrieving articles from page 2394
Retrieving articles from page 2395
Retrieving articles from page 2396
Retrieving articles from page 2397
Retrieving articles from page 2398
Retrieving articles from page 2399
Retrieving articles from page 2400
Retrieving articles from page 2401
Retrieving articles from page 2402
Retrieving articles from page 2403
Retrieving articles from page 2404
Retrieving articles from page 2405
Retrieving articles 

Retrieving articles from page 2613
Retrieving articles from page 2614
Retrieving articles from page 2615
Retrieving articles from page 2616
Retrieving articles from page 2617
Retrieving articles from page 2618
Retrieving articles from page 2619
Retrieving articles from page 2620
Retrieving articles from page 2621
Retrieving articles from page 2622
Retrieving articles from page 2623
Retrieving articles from page 2624
Retrieving articles from page 2625
Retrieving articles from page 2626
Retrieving articles from page 2627
Retrieving articles from page 2628
Retrieving articles from page 2629
Retrieving articles from page 2630
Retrieving articles from page 2631
Retrieving articles from page 2632
Retrieving articles from page 2633
Retrieving articles from page 2634
Retrieving articles from page 2635
Retrieving articles from page 2636
Retrieving articles from page 2637
Retrieving articles from page 2638
Retrieving articles from page 2639
Retrieving articles from page 2640
Retrieving articles 

Retrieving articles from page 2848
Retrieving articles from page 2849
Retrieving articles from page 2850
Retrieving articles from page 2851
Retrieving articles from page 2852
Retrieving articles from page 2853
Retrieving articles from page 2854
Retrieving articles from page 2855
Retrieving articles from page 2856
Retrieving articles from page 2857
Retrieving articles from page 2858
Retrieving articles from page 2859
Retrieving articles from page 2860
Retrieving articles from page 2861
Retrieving articles from page 2862
Retrieving articles from page 2863
Retrieving articles from page 2864
Retrieving articles from page 2865
Retrieving articles from page 2866
Retrieving articles from page 2867
Retrieving articles from page 2868
Retrieving articles from page 2869
Retrieving articles from page 2870
Retrieving articles from page 2871
Retrieving articles from page 2872
Retrieving articles from page 2873
Retrieving articles from page 2874
Retrieving articles from page 2875
Retrieving articles 

In [59]:
# find index of journal
i = 0
for journal in journals:
    if journal[0] == 'Observatorio (OBS*)':
        print(i)
    i +=1

7441


In [60]:
len(journals)

10352